# Imports,  Settings and Paths

In [1]:
comment = 'new_arch_no_lr_scheduler'

In [2]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import torch
import torch.utils.data as data_utils

In [3]:
from Code.Models.c7o2h10_model import DeepPotential, normalize
from Code.Models.nn_extentions import AbcExponentialLR

In [4]:
data_path = './Dataset/c7o2h10_X.npy'
label_path = './Dataset/c7o2h10_Y.npy'
train_ids_path = './Dataset/iso17/train_ids.txt'
test_ids_path = './Dataset/iso17/validation_ids.txt'
model_path = './ModelCheckpoints/c7o2h10/{}/'.format(comment)

In [5]:
use_cuda = torch.cuda.is_available()
print('use cuda:  ', use_cuda)

use cuda:   True


# Read the Data

In [6]:
train_ids = (np.loadtxt(train_ids_path) - 1).astype(int).tolist()
test_ids = (np.loadtxt(test_ids_path) - 1).astype(int).tolist()

In [7]:
X_npy = np.load(data_path)
Y_npy = np.load(label_path) * -1

### Normalize Y Data

In [8]:
Y_npy, Y_min, Y_max = normalize(Y_npy)
X_npy.shape

(404000, 19, 72)

### Standardize X Data

In [9]:
for column in range(X_npy.shape[-1]):
    col_mean = X_npy[:, :, column].mean()
    col_std = X_npy[:, :, column].std()
    X_npy[:, :, column] = (X_npy[:, :, column] - col_mean) / col_std

**test if mean is small:**

In [10]:
X_npy[:, :, 0].mean()#, X_npy.std(axis=2)

-8.620739233346562e-15

**test if std is nearly 1:**

In [11]:
X_npy[:, :, 0].std()

0.9999999999999996

### Check for NaN values

In [12]:
use_ids = np.where(np.isnan(X_npy).sum(axis=2).sum(axis=1)==0)
print('{} Datapoints with NaN values found.'.format(X_npy.shape[0] - len(use_ids[0])))

0 Datapoints with NaN values found.


In [13]:
X_npy = X_npy[use_ids]
Y_npy = Y_npy[use_ids]
X_npy.shape

(404000, 19, 72)

In [14]:
X_npy = X_npy
Y_npy = Y_npy

In [23]:
X_npy.shape

(404000, 19, 72)

## Shuffle

In [15]:
ids = np.arange(0, Y_npy.shape[0])
np.random.shuffle(ids)
X_npy = X_npy[ids]
Y_npy = Y_npy[ids]

### Split into test and train

In [16]:
use_for_test = 80000
use_for_train = X_npy.shape[0] - use_for_test
X_train_npy, X_test_npy = X_npy[:use_for_train], X_npy[use_for_train:]
Y_train_npy, Y_test_npy = Y_npy[:use_for_train], Y_npy[use_for_train:]


In [17]:
#X_train_npy, X_test_npy = X_npy[train_ids], X_npy[test_ids]
#Y_train_npy, Y_test_npy = Y_npy[train_ids], Y_npy[test_ids]
print('X Train shape: {}\tX Test shape: {}\nY Train shape: {}\tY Test shape: {}'.format(X_train_npy.shape,
                                                                                        X_test_npy.shape,
                                                                                        Y_train_npy.shape,
                                                                                        Y_test_npy.shape))

X Train shape: (324000, 19, 72)	X Test shape: (80000, 19, 72)
Y Train shape: (324000,)	Y Test shape: (80000,)


### Create Pytorch Variables

In [18]:
#use_x_values = 100000
#X_train_npy = X_train_npy[:use_x_values]
#Y_train_npy = Y_train_npy[:use_x_values]

In [19]:
X_train_npy.shape

(324000, 19, 72)

In [20]:
train_dataset = data_utils.TensorDataset(torch.DoubleTensor(X_train_npy), torch.DoubleTensor(Y_train_npy))
train_loader = data_utils.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True, pin_memory=True)
test_dataset = data_utils.TensorDataset(torch.DoubleTensor(X_test_npy), torch.DoubleTensor(Y_test_npy))
test_loader = data_utils.DataLoader(test_dataset, batch_size=128, drop_last=True, pin_memory=True)

# Train Model

## Do the learning

In [21]:
deep_pot = DeepPotential(use_cuda=use_cuda,
                         checkpoint_path=model_path)

/home/mi/stefaah94/miniconda3/lib/python3.6/site-packages/torch/cuda/__init__.py:116: UserWarning: 
    Found GPU1 NVS 310 which is of cuda capability 2.1.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [22]:
train_losses, test_losses = deep_pot.fit(train_loader=train_loader, n_epochs=300, test_loader=test_loader)

epoch: 1	progress: 0.0	time estimate: 275.63	train loss: 0.191017	test loss: 0.190819
epoch: 2	progress: 0.01	time estimate: 261.55	train loss: 0.190736	test loss: 0.190819
epoch: 3	progress: 0.01	time estimate: 247.57	train loss: 0.19074	test loss: 0.190819


KeyboardInterrupt: 

In [ ]:
test_losses

In [ ]:
plt.plot(train_losses)
plt.plot(test_losses)
plt.semilogy()

In [ ]:
for x, y in train_loader:
    print(deep_pot.cpu()(x)[:10], y[:10])
    break

## Load Model

In [ ]:
test_model = DeepPotential().cuda()
test_model.load_state_dict(torch.load('ModelCheckpoints/c7o2h10/epoch_280'))

In [ ]:
test_result = np.squeeze(backtransform(test_model(X_test), Y_min, Y_max).detach().cpu().numpy())

In [ ]:
val_result = np.squeeze(backtransform(Y_test, Y_min, Y_max).detach().cpu().numpy())

In [ ]:
np.mean(np.abs(test_result - val_result))

In [ ]:
val_result[:10]

## Test the model
### Mean Absolute Error
The desired accuracy is about 0.04 eV

In [ ]:
mae = np.abs(deep_pot.forward(X_data[-500:]).data.numpy().reshape(500) - Y_data[-500:].data.numpy().reshape(500)).mean()
print('The nural network reaches a mean absolute error of {} eV'.format(mae))

### Small test sample

In [ ]:
deep_pot.forward(X_data[-10:])

In [ ]:
Y_data[-10:]

## Save the Model Parameters

In [ ]:
torch.save(deep_pot.state_dict(), model_path)

## Continue learning

In [ ]:
deep_pot = DeepPotential()
deep_pot.load_state_dict(torch.load(model_path))
optim = torch.optim.Adam(deep_pot.parameters(), lr=1e-2)

In [ ]:
deep_pot, optim = train(deep_pot, optim, X_data, Y_data, 1000, 512, (0.001, 0.96, 1.5), use_for_train=0.9, print_every=10)